### Importación de datos



In [2]:
import pandas as pd

url = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_1%20.csv"
url2 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_2.csv"
url3 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_3.csv"
url4 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_4.csv"

tienda = pd.read_csv(url)
tienda2 = pd.read_csv(url2)
tienda3 = pd.read_csv(url3)
tienda4 = pd.read_csv(url4)

tienda.head()

,Producto,Categoría del Producto,Precio,Costo de envío,Fecha de Compra,Vendedor,Lugar de Compra,Calificación,Método de pago,Cantidad de cuotas,lat,lon
0,Asistente virtual,Electrónicos,164300.0,6900.0,16/01/2021,Pedro Gomez,Bogotá,4,Tarjeta de crédito,8,4.60971,-74.08175
1,Mesa de comedor,Muebles,192300.0,8400.0,18/05/2022,Beatriz Morales,Medellín,1,Tarjeta de crédito,4,6.25184,-75.56359
2,Juego de mesa,Juguetes,209600.0,15900.0,15/03/2021,Juan Fernandez,Cartagena,1,Tarjeta de crédito,1,10.39972,-75.51444
3,Microondas,Electrodomésticos,757500.0,41000.0,03/05/2022,Juan Fernandez,Cali,4,Nequi,1,3.43722,-76.52250
4,Silla de oficina,Muebles,335200.0,20200.0,07/11/2020,Maria Alfonso,Medellín,5,Nequi,1,6.25184,-75.56359


#1. Análisis de facturación



In [ ]:
# Facturación total por tienda (suma del Precio de todas las ventas)
facturacion_tienda_1 = tienda['Precio'].sum()
facturacion_tienda_2 = tienda2['Precio'].sum()
facturacion_tienda_3 = tienda3['Precio'].sum()
facturacion_tienda_4 = tienda4['Precio'].sum()

resumen_facturacion = pd.DataFrame({
    'Tienda': ['Tienda 1', 'Tienda 2', 'Tienda 3', 'Tienda 4'],
    'Facturación Total': [facturacion_tienda_1, facturacion_tienda_2, facturacion_tienda_3, facturacion_tienda_4]
})
resumen_facturacion['Facturación Total'] = resumen_facturacion['Facturación Total'].apply(lambda x: f"${x:,.0f}")
print("=== FACTURACIÓN TOTAL POR TIENDA ===")
resumen_facturacion

In [ ]:
# Gráfico de facturación total
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 4))
tiendas = ['Tienda 1', 'Tienda 2', 'Tienda 3', 'Tienda 4']
totales = [tienda['Precio'].sum(), tienda2['Precio'].sum(), tienda3['Precio'].sum(), tienda4['Precio'].sum()]
bars = ax.bar(tiendas, totales, color=['#2ecc71', '#3498db', '#9b59b6', '#e74c3c'])
ax.set_ylabel('Facturación total ($)')
ax.set_title('Facturación total por tienda')
for bar in bars:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'${bar.get_height()/1e6:.1f}M', ha='center', va='bottom', fontsize=9)
plt.tight_layout()
plt.show()

# 2. Ventas por categoría

In [ ]:
# Categorías más populares (por cantidad de ventas y por facturación) en cada tienda
def top_categorias(df, nombre_tienda):
    ventas_cat = df.groupby('Categoría del Producto').agg(
        Ventas=('Precio', 'count'),
        Facturación=('Precio', 'sum')
    ).sort_values('Ventas', ascending=False)
    ventas_cat['Tienda'] = nombre_tienda
    return ventas_cat.reset_index()

cat_t1 = top_categorias(tienda, 'Tienda 1')
cat_t2 = top_categorias(tienda2, 'Tienda 2')
cat_t3 = top_categorias(tienda3, 'Tienda 3')
cat_t4 = top_categorias(tienda4, 'Tienda 4')

print("=== CATEGORÍAS MÁS POPULARES (por número de ventas) ===\n")
for name, df_cat in [('Tienda 1', cat_t1), ('Tienda 2', cat_t2), ('Tienda 3', cat_t3), ('Tienda 4', cat_t4)]:
    print(f"{name}: {df_cat['Categoría del Producto'].iloc[0]} ({df_cat['Ventas'].iloc[0]} ventas)")
cat_t1.head(10)

In [ ]:
# Resumen categorías todas las tiendas
pd.concat([cat_t1, cat_t2, cat_t3, cat_t4]).pivot_table(
    index='Categoría del Producto', columns='Tienda', values='Ventas', aggfunc='sum'
).fillna(0).astype(int)

# 3. Calificación promedio de la tienda


In [ ]:
# Promedio de facturación por cliente (cada fila = una venta, tomamos promedio de Precio por tienda = ticket promedio)
promedio_fact_cliente = pd.DataFrame({
    'Tienda': ['Tienda 1', 'Tienda 2', 'Tienda 3', 'Tienda 4'],
    'Promedio facturación por venta (ticket promedio)': [
        tienda['Precio'].mean(),
        tienda2['Precio'].mean(),
        tienda3['Precio'].mean(),
        tienda4['Precio'].mean()
    ],
    'Nº de ventas': [len(tienda), len(tienda2), len(tienda3), len(tienda4)]
})
promedio_fact_cliente['Promedio facturación por venta (ticket promedio)'] = promedio_fact_cliente['Promedio facturación por venta (ticket promedio)'].apply(lambda x: f"${x:,.0f}")
print("=== PROMEDIO DE FACTURACIÓN POR VENTA (TICKET PROMEDIO) ===")
promedio_fact_cliente

In [ ]:
# Gráfico ticket promedio
fig, ax = plt.subplots(figsize=(8, 4))
promedios = [tienda['Precio'].mean(), tienda2['Precio'].mean(), tienda3['Precio'].mean(), tienda4['Precio'].mean()]
ax.bar(tiendas, promedios, color=['#2ecc71', '#3498db', '#9b59b6', '#e74c3c'])
ax.set_ylabel('Ticket promedio ($)')
ax.set_title('Promedio de facturación por venta por tienda')
plt.tight_layout()
plt.show()

# 4. Productos más y menos vendidos

In [ ]:
# Productos más y menos vendidos (por cantidad de ventas = veces que aparece cada producto)
def productos_vendidos(df, nombre_tienda):
    prod = df.groupby('Producto').agg(Ventas=('Precio', 'count'), Facturación=('Precio', 'sum')).sort_values('Ventas', ascending=False)
    prod['Tienda'] = nombre_tienda
    return prod.reset_index()

p_t1 = productos_vendidos(tienda, 'Tienda 1')
p_t2 = productos_vendidos(tienda2, 'Tienda 2')
p_t3 = productos_vendidos(tienda3, 'Tienda 3')
p_t4 = productos_vendidos(tienda4, 'Tienda 4')

print("=== PRODUCTOS MÁS VENDIDOS (top 3 por tienda) ===")
for name, df_p in [('Tienda 1', p_t1), ('Tienda 2', p_t2), ('Tienda 3', p_t3), ('Tienda 4', p_t4)]:
    top3 = df_p.nlargest(3, 'Ventas')[['Producto', 'Ventas']]
    print(f"\n{name}:")
    print(top3.to_string(index=False))

print("\n=== PRODUCTOS MENOS VENDIDOS (bottom 3 por tienda) ===")
for name, df_p in [('Tienda 1', p_t1), ('Tienda 2', p_t2), ('Tienda 3', p_t3), ('Tienda 4', p_t4)]:
    bot3 = df_p.nsmallest(3, 'Ventas')[['Producto', 'Ventas']]
    print(f"\n{name}:")
    print(bot3.to_string(index=False))

In [ ]:
# Tabla resumen: más vendido y menos vendido por tienda
resumen_productos = []
for name, df_p in [('Tienda 1', p_t1), ('Tienda 2', p_t2), ('Tienda 3', p_t3), ('Tienda 4', p_t4)]:
    mas = df_p.loc[df_p['Ventas'].idxmax()]
    menos = df_p.loc[df_p['Ventas'].idxmin()]
    resumen_productos.append({'Tienda': name, 'Más vendido': mas['Producto'], 'Ventas (más)': mas['Ventas'], 'Menos vendido': menos['Producto'], 'Ventas (menos)': menos['Ventas']})
pd.DataFrame(resumen_productos)

# 5. Envío promedio por tienda

In [ ]:
# Costo promedio del envío por tienda
costo_envio = pd.DataFrame({
    'Tienda': ['Tienda 1', 'Tienda 2', 'Tienda 3', 'Tienda 4'],
    'Costo promedio envío': [
        tienda['Costo de envío'].mean(),
        tienda2['Costo de envío'].mean(),
        tienda3['Costo de envío'].mean(),
        tienda4['Costo de envío'].mean()
    ],
    'Costo total envíos': [
        tienda['Costo de envío'].sum(),
        tienda2['Costo de envío'].sum(),
        tienda3['Costo de envío'].sum(),
        tienda4['Costo de envío'].sum()
    ]
})
costo_envio['Costo promedio envío ($)'] = costo_envio['Costo promedio envío'].apply(lambda x: f"${x:,.0f}")
print("=== COSTO PROMEDIO DEL ENVÍO POR TIENDA ===")
costo_envio[['Tienda', 'Costo promedio envío ($)', 'Costo total envíos']]

In [ ]:
# Gráfico costo promedio envío
fig, ax = plt.subplots(figsize=(8, 4))
costos = [tienda['Costo de envío'].mean(), tienda2['Costo de envío'].mean(), tienda3['Costo de envío'].mean(), tienda4['Costo de envío'].mean()]
ax.bar(tiendas, costos, color=['#2ecc71', '#3498db', '#9b59b6', '#e74c3c'])
ax.set_ylabel('Costo promedio envío ($)')
ax.set_title('Costo promedio de envío por tienda')
plt.tight_layout()
plt.show()

# 6. Reporte: ¿Qué tienda vender? (Recomendación para el Señor Juan)

In [ ]:
# Resumen numérico para la decisión (todas las métricas en valores numéricos)
fact_total_t1 = tienda['Precio'].sum()
fact_total_t2 = tienda2['Precio'].sum()
fact_total_t3 = tienda3['Precio'].sum()
fact_total_t4 = tienda4['Precio'].sum()

ticket_t1 = tienda['Precio'].mean()
ticket_t2 = tienda2['Precio'].mean()
ticket_t3 = tienda3['Precio'].mean()
ticket_t4 = tienda4['Precio'].mean()

envio_t1 = tienda['Costo de envío'].mean()
envio_t2 = tienda2['Costo de envío'].mean()
envio_t3 = tienda3['Costo de envío'].mean()
envio_t4 = tienda4['Costo de envío'].mean()

n_ventas_t1 = len(tienda)
n_ventas_t2 = len(tienda2)
n_ventas_t3 = len(tienda3)
n_ventas_t4 = len(tienda4)

reporte = pd.DataFrame({
    'Tienda': ['Tienda 1', 'Tienda 2', 'Tienda 3', 'Tienda 4'],
    'Facturación total': [fact_total_t1, fact_total_t2, fact_total_t3, fact_total_t4],
    'Nº ventas': [n_ventas_t1, n_ventas_t2, n_ventas_t3, n_ventas_t4],
    'Ticket promedio': [ticket_t1, ticket_t2, ticket_t3, ticket_t4],
    'Costo prom. envío': [envio_t1, envio_t2, envio_t3, envio_t4]
})
# Margen aproximado: facturación menos costos de envío (como proxy de rentabilidad)
reporte['Facturación neta (aprox.)'] = reporte['Facturación total'] - reporte['Nº ventas'] * reporte['Costo prom. envío']
reporte

In [ ]:
# Identificar la tienda MENOS lucrativa (menor facturación total y/o peores indicadores)
# Criterio: menor facturación total = menos lucrativa para vender y liberar capital
tienda_menos_facturacion = reporte.loc[reporte['Facturación total'].idxmin(), 'Tienda']
menor_fact = reporte['Facturación total'].min()
ranking = reporte.sort_values('Facturación total', ascending=True)[['Tienda', 'Facturación total', 'Nº ventas', 'Ticket promedio']]
ranking['Facturación total'] = ranking['Facturación total'].apply(lambda x: f"${x:,.0f}")
ranking['Ticket promedio'] = reporte.sort_values('Facturación total', ascending=True)['Ticket promedio'].apply(lambda x: f"${x:,.0f}")
print("Ranking de tiendas por facturación total (de menor a mayor):")
ranking

In [ ]:
print("=" * 70)
print("  REPORTE EJECUTIVO: RECOMENDACIÓN PARA EL SEÑOR JUAN")
print("=" * 70)
print()
print(f"  TIENDA RECOMENDADA PARA VENDER (menos lucrativa): {tienda_menos_facturacion}")
print()
print("  Resumen de análisis:")
print("  • Facturación total: la tienda con menor facturación genera menos ingresos.")
print("  • Categorías populares: varían por tienda; la menos lucrativa suele tener")
print("    menor volumen de ventas en categorías clave.")
print("  • Ticket promedio: indica cuánto gasta en promedio cada venta.")
print("  • Productos más/menos vendidos: reflejan la demanda por tienda.")
print("  • Costo promedio de envío: impacta la rentabilidad neta.")
print()
print("  Conclusión: Vender la tienda con MENOR facturación total permite liberar")
print("  recursos para el nuevo negocio con el menor impacto en los ingresos actuales.")
print("=" * 70)